In [1]:
import findspark
# $ cd /opt
# /opt$ sudo ln -s ~/apps/spark-3.4.0-bin-hadoop3 spark
findspark.init("/opt/spark") 

In [2]:
# https://github.com/GoogleCloudDataproc/hadoop-connectors/releases/

# ! wget -P ~/apps https://github.com/GoogleCloudDataproc/hadoop-connectors/releases/download/v2.2.13/gcs-connector-hadoop2-2.2.13-shaded.jar

In [3]:
# from pathlib import Path
# gcs_jar = str(Path('~/apps/gcs-connector-hadoop2-2.2.13-shaded.jar').expanduser().resolve())

# from pyspark.sql import SparkSession
# spark = SparkSession.builder \
#             .appName("LensFeatures") \
#             .config("spark.jars", gcs_jar) \
#             .config("spark.jars.package", gcs_jar) \
#             .getOrCreate()

In [4]:
# conf = spark.sparkContext._jsc.hadoopConfiguration()
# conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
# conf.set("fs.AbstractFileSystem.gs.impl","com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
# conf.set("fs.gs.project.id", "boxwood-well-386122")
# conf.set("fs.gs.auth.service.account.enable", "true")
# conf.set("fs.gs.auth.service.account.json.keyfile", secretLocation)

In [5]:
# ! gcloud auth application-default login

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .appName("LensFeatures") \
            .config("spark.sql.session.timeZone", "UTC")
            .getOrCreate()

23/05/18 11:15:59 WARN Utils: Your hostname, VIJAYs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.87.240 instead (on interface en0)
23/05/18 11:15:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 11:16:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [77]:
# df = spark.read.option("header",True).parquet("gs://vijay-lens-bigquery-export/public_profile/")
# df = spark.read.format("parquet").load("gs://vijay-lens-bigquery-export/public_profile/")
df = spark.read \
            .parquet("/tmp/partitioned/")

In [78]:
df.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- handle: string (nullable = true)
 |-- profile_with_weights: string (nullable = true)
 |-- profile_picture_s3_url: string (nullable = true)
 |-- cover_picture_s3_url: string (nullable = true)
 |-- is_profile_picture_onchain: boolean (nullable = true)
 |-- owned_by: string (nullable = true)
 |-- is_burnt: boolean (nullable = true)
 |-- nft_contract: string (nullable = true)
 |-- nft_token_id: string (nullable = true)
 |-- nft_owner: string (nullable = true)
 |-- nft_chain_id: long (nullable = true)
 |-- is_default: boolean (nullable = true)
 |-- metadata_url: string (nullable = true)
 |-- metadata_s3_url: string (nullable = true)
 |-- is_metadata_processed: boolean (nullable = true)
 |-- has_error: boolean (nullable = true)
 |-- timeout_request: boolean (nullable = true)
 |-- metadata_error_reason: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- metadata_block_timestamp: timestamp_

In [79]:
# "bq extract" and "pyspark" write timestamps differently
# reading parquet files generated by the two methods causes conflicts 
# casting doesn't help. So, drop the columns for now
# df = df.withColumn("block_timestamp", df.block_timestamp.cast("timestamp"))
# df = df.withColumn("metadata_block_timestamp", df.block_timestamp.cast("timestamp"))


from pyspark.sql.functions import col
df = df.drop(col("datastream_metadata")) \
        .drop(col("block_timestamp")) \
        .drop(col("metadata_block_timestamp"))

In [12]:
from datetime import datetime, timezone
prev_checkpoint = "2023-05-10 00:00:00 UTC"
dt = datetime.strptime(prev_checkpoint, "%Y-%m-%d %H:%M:%S %Z")
dt

datetime.datetime(2023, 5, 10, 0, 0)

In [13]:
condn = dt.strftime("%Y%m%d%H%M%S")
condn

'20230510000000'

In [80]:
from pyspark.sql.functions import max, to_utc_timestamp

df.where(f"dtime > {condn}").show(10, truncate=False)

+----------+---------+---------------------+----------------------------------+-----------------------------------------------------+--------------------+--------------------------+------------------------------------------+--------+------------+------------+---------+------------+----------+---------------------------------------------------------------+--------------------------------------------------------------------------------------+---------------------+---------+---------------+---------------------+------+------------------------------------------------------------------+------------------------------------------------------------------+------------------------------------------------------------------+--------------+
|profile_id|name     |handle               |profile_with_weights              |profile_picture_s3_url                               |cover_picture_s3_url|is_profile_picture_onchain|owned_by                                  |is_burnt|nft_contract|nft_token_id|nft